In [ ]:
Make sure your credential is ready before running this script!

In [1]:
import tweepy
from datetime import datetime, timedelta
from dateutil.parser import parse
import json
import yaml
from bson import json_util
from time import sleep
import os

def authentication(credential):
    print('Authenticating...', end='\r')
    auth = tweepy.OAuthHandler(credential['CONSUMER_KEY'], credential['CONSUMER_SECRET'])
    auth.set_access_token(credential['ACCESS_TOKEN'], credential['ACCESS_TOKEN_SECRET'])
    apitweepy = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    print('Authentication success using account : @{}'.format(apitweepy.auth.get_username()))
    return apitweepy

def correcting_time_local7(since, until):
    since = (parse(since) - timedelta(days=1)).strftime('%Y-%m-%d')
    until = (parse(until) + timedelta(days=1)).strftime('%Y-%m-%d')
    return since, until

def search_for_tweet(apitweepy, topic, search, language, since, until):
    result_path = 'tweet_search_result/{}/'.format(topic)
    if not os.path.exists(result_path):
        os.makedirs(result_path)

    engine = tweepy.Cursor(apitweepy.search, 
                       q='"{}"'.format(search),
                       lang=language,
                       since=since,
                       until=until,
                       count=100,
                       result_type='recent',
                       tweet_mode='extended', 
                       include_entities=True)

    min_date = datetime.now()
    result_counter = 0
    since_out = (parse(since) + timedelta(days=1)).strftime('%Y%m%d')
    until_out = (parse(until) - timedelta(days=1)).strftime('%Y%m%d')
    with open('{}search_tweet_{}_{}-{}.json'.format(result_path, search, since_out, until_out), 'w+') as f:
        while True:
            try:
                for tweets in engine.pages():
                    tweets = [i._json for i in tweets]
                    for i in tweets:
                        local_date = (parse(i['created_at'], ignoretz=True) + timedelta(hours=7))
                        i.update({'created_at_local': local_date.strftime('%Y-%m-%d %H:%M:%S')})
                        if local_date < min_date:
                            min_date = local_date
                        if min_date < parse(since):
                            break
                        else:
                            result_counter+=1
                            f.write(json.dumps(i, default=json_util.text_type)+'\n')
                            f.flush()
                    print('{} |INFO| {} {} tweets collected'.format(min_date, search, result_counter), end='\r')
                break
            except tweepy.error.TweepError:
                sleep(5)
        print('{} |INFO| {} {} tweets collected'.format(min_date, search, result_counter))
        f.close()

In [2]:
if __name__ == '__main__':
    credential = yaml.load(open('auth_twitter.yaml'), Loader=yaml.Loader)
    apitweepy = authentication(credential)
    since = input('since date [YYYY-MM-DD]: ')
    until = input('until date [YYYY-MM-DD]: ')
    since, until = correcting_time_local7(since, until)
    language = input('language code: ')
    topic = input('topic name: ')
    print('*this script only support keyword that using one word')
    print('*and search method is exact match')
    search = input('search keywords [separate keyword with "***"]: ')
    for s in search.split('***'):
        search_for_tweet(apitweepy, topic, s, language, since, until)

Authentication success using account : @fchrulk
since date [YYYY-MM-DD]: 2019-09-01
until date [YYYY-MM-DD]: 2019-09-02
language code: id
topic name: new bawaslu
*this script only support keyword that using one word
*and search method is exact match
search keywords [separate keyword with "***"]: bawaslu***pemilu
2019-08-31 07:05:30 |INFO| bawaslu 1233 tweets collected
2019-08-31 07:00:57 |INFO| pemilu 4695 tweets collected


In [ ]:
Wait until it finished!
It may takes long time depends how many users that used the keyword on their tweets
JSON output should be created on your work directory.
Sometime rate limit exceeded, just wait, it will be done.